55. 跳跃游戏

https://leetcode.cn/problems/jump-game/description/?envType=study-plan-v2&envId=top-interview-150

在用贪心算法做之前，让我们先用比较暴力的 DFS 算法做一遍。该算法最好理解，但是我最开始想的其实还僵化在找不等式约束，如下是我写的约束条件
- 存在元素 x1, x2, x3 ... 使得满足局部条件，即如 r(x1) 确实能够到 x2；
- 整体满足全局条件，即它们作为跳板，最终可以到达 -1 位置，x1 + x2 + x3 + ... > len(nums)。

但是实际还是需要**搜索**，也就是动态地确定问题的答案，不可能是一个整体前向的过程。先前做了太多列表的题，都是需要构建前向或者双向的循环解决问题，其过程是**非递归的**。

DFS 深度优先搜索，是存在标准套路的。
首先，写一个 dfs()，然后确定以下关键组件
- 输入，输入变量是什么：简单来说，就是这个搜索树的指针，也是在递归过程中要传递下去的变量，这里就是数组位置 i 了；
- 返回值：返回值要第一时间想到最终希望返回什么，本题自然是 True or False；第二时间要想的是回传过程的信息累加，也就是说当搜索到某个节点符合某种要求时，返回的路径上是否需要进一步收集信息，例如累加，本题不需要；
- 中间内容：整体分两部分，第一部分是对当前节点判断；第二部分是探索下一节点的动作，通常是遍历叶子节点，并 catch 它们的返回值，如果需要累加信息就对应地写好逻辑过程；当然，最后如果遍历一遍当前节点的子节点后没有找到，需要返回对应的 False 信息。
- 最后，整体的代码应该给一个入口，如 dfs(0)。

该算法的时间复杂度很高，最坏情况指数级，空间复杂度为 o(1)。

In [ ]:
class Solution:
    def canJump(self, nums: List[int]) -> bool:
        n = len(nums)
        
        def dfs(i):
            if i == n - 1: # larger than or equal to?
                return True
            for j in range(i + 1, min(n - 1, i + nums[i]) + 1): # + 1
                if dfs(j):
                    return True
            return False
        
        return dfs(0)

注意到当某个元素的值是 0 时，range(i + 1, min(n - 1, i + nums[i]) + 1) 将退化为 range(i + 1, i + 1)，其范围是 0，不会继续探索。这是符合要求的。

In [4]:
for i in range(5, 5):
    print(i)

对于 DFS 等搜索算法，如果想要主动避免重复节点的搜索，且每次搜索 a 对于结果 y 是独立的，即 p(y|a1) 独立于 p(y|a2)，那么有一个非常简单易懂的补丁式算法，即加一个 memo 缓存，记忆已经遍历过的节点的 p(y|a) 即可。如下。该算法的时间复杂度实际上不是 o(n) 而是 o(n^2)，因为实际上对于某个已经进入的节点，我们还是会依次遍历其子节点（多一层深度，对于 n^2），只不过不会再多一层，对于已经探索的节点。

In [ ]:
class Solution:
    def canJump(self, nums: List[int]) -> bool:
        n = len(nums)
        memo = [None] * n

        def dfs(i: int) -> bool:
            if i >= n - 1:
                return True
            if memo[i] is not None:
                return memo[i]

            furthest = min(n - 1, i + nums[i])
            for j in range(i + 1, furthest + 1):
                if dfs(j):
                    memo[i] = True
                    return True
            memo[i] = False
            return False

        return dfs(0)

虽然，问题不是直接可以前向遍历的，而是要借助递归解决；但是，树搜索有一个对偶存在，即著名的 DP 动态规划！动态规划是一种有状态（stateful）的遍历，也就是，当前遍历的计算是基于先前遍历的结果的。

In [ ]:
class Solution:
    def canJump(self, nums: List[int]) -> bool:
        n = len(nums)
        dp = [False] * n
        dp[0] = True

        for i in range(1, n):
            for j in range(i):
                if dp[j] and j + nums[j] >= i:
                    dp[i] = True
                    break

        return dp[n - 1]

可以看到，这里的核心条件就是，if dp[j] and j + nums[j] >= i。dp 的核心是确定 dp 到底在维护什么，确定的方法也很简单，题目求什么我们就维护什么，本题想要知道 start 能否 reach 到 end；那么 dp 维护的就是当前这一轮次下，start 能否 reach 到每个元素的状态。直到 dp 结束时，就能知道 start 能否 reach 到 end。时间复杂度是 o(n^2) 的。

从 DFS 的树搜索（指数级别），到有记录状态的 stateful DFS，再到 stateful DP 也即动态规划（注意这里二者是一种对偶，一个以当前节点为探索单位，看它能走多远，一个以当前节点为利用单位，看 start 能否到达它），最后终于来到 greedy 贪心算法。

In [6]:
from typing import List

class Solution:
    def canJump(self, nums: List[int]) -> bool:
        n = len(nums)
        max_reach = 0
        for i in range(n):
            if i > max_reach:
                return False
            max_reach = max(max_reach, i + nums[i])
            if max_reach >= n - 1:
                return True

确实不好想，但是掌握方法论之后更好理解，以及理解为什么能把时间复杂度降低到 o(n)，利用了何样的信息？

第一个信息点在于，对于第 k 轮，我们其实只关心 dp[k]，但是 dp[k] 的计算却依赖于 dp[1] 到 dp[k - 1]，因此我们开了一个很大的序列来存储这些信息。如果不用呢？实际上，dp[k] 是否为 True 的信息，早就蕴藏在 dp[k - m] 和 (nums[k - m] + (k - m) > k) 中了；如果反复执行这一回溯操作并展开，我们可以得到，存在 0 < i1 < i2 ... < k，有
```
dp[k] = dp[0] & (nums[i1] + i1 > i2) & (nums[i2] + i2 > i3) & ...
      = dp[0] & (nums[i1] + i1 - i2 > 0) & (nums[i2] + i2 - i3 > 0) & ...
      = (nums[0] + 0 - 0 > 0) & (nums[i1] + i1 - i2 > 0) & (nums[i2] + i2 - i3 > 0) & ...
```
到这一步，我们推导出，每一次遍历都是在做概率乘，或者说是概率链式计算，即 p(s) * p(a,s) 大概意思如此。也就是说，新引入的信息量是，**如果当前遍历引入了一个新的元素，那么这个元素是否能让起点走的更远？**

显而易见地，该变量是闭包的，也即 max_reach 如果已经小于 i 时，说明已经超出范围了，中断即可；而如果还大于 i 时，说明此时该变量还可以引入新的能量。

所以说贪心实际上就是带状态的，或者说递归的最优化问题。我们想解决的是 
`max_reach = max(max_reach, i + nums[i])`，这就是一个贪心问题。

45. 跳跃游戏 II

https://leetcode.cn/problems/jump-game-ii/description/?envType=study-plan-v2&envId=top-interview-150

如下还是先给出一个基于 dp 的做法，可以看到相比于之前的 dp 是传递记录 True or False，如下 dp 则改为记录 dp 的最小跳。

In [14]:
class Solution:
    def jump(self, nums: List[int]) -> int:
        n = len(nums)
        dp = [float("inf")] * n
        dp[0] = 0
        for i in range(1, n):
            for j in range(i): # be careful
                if nums[j] + j >= i:
                    dp[i] = min(dp[i], dp[j] + 1)
        return dp[n - 1]

In [15]:
solution = Solution()
solution.jump(nums=[2,3,1,1,4])

2

同样地，我们

121. 买卖股票的最佳时机

https://leetcode.cn/problems/best-time-to-buy-and-sell-stock/description/?envType=study-plan-v2&envId=top-interview-150

股票买卖问题的本质即，对于数组中的所有元素，想找到两个元素，满足
- num[i] - num[j] 这一 metric 最大，即 max(num[i] - num[j])；
- i 必须在 j 之后，即 i > j;

这是一个标准的最优化目标，一个目标函数，一个 constraint；不论这两个事情怎么换（比如，改成 max(num[i] * num[j])，或者改成 i > j），其思路是不变的。

In [ ]:
class Solution:
    def maxProfit(self, prices: List[int]) -> int:
        n, best = len(prices), 0
        if n < 2:
            return best
        for i in range(n):
            for j in range(i + 1, n):
                diff = prices[j] - prices[i]
                best = max(diff, best)
        return best

如上自然是最好想的写法，也就是 dp 了，非常直觉明了，但是时间复杂度是二阶的。那么更好的做法是什么？这里要反思的是，哪些操作是冗余的，很明显，我们对于元素的访问是冗余的，也就是说访问的次数**本可以更少的**。

我们现在要做的变换是，将以 max(num[i] - num[j]) 和 i > j 为目标的 (i, j) 遍历方法，改为只遍历 i，并基于新的变量记录 min(num[:i])，在数学上是一种将约束条件放回到目标函数的做法，也就是贪心。实际上，就是维护“到目前为止的最低买入价”，并实时更新最大利润。如下

$$
\max \left( num[i] - num[j] \right) \quad \text{s.t.} \quad i > j \\
= \max (num[i] - \min(num[:i])) \quad \forall\, i > 0
$$

可以看到，相比于如上的贪心，这里的贪心是在动态维护最大差额利润，而上一类题是在动态维护最远距离。

In [ ]:
from typing import List

class Solution:
    def maxProfit(self, prices: List[int]) -> int:
        min_price = float("inf")
        max_profit = 0

        for p in prices:
            if p < min_price:
                min_price = p
            else:
                max_profit = max(max_profit, p - min_price)

        return max_profit

或者如下，在实现上只有一次更新的相位差。但要学会正无穷的写法，即 float("inf")。

In [ ]:
class Solution:
    def maxProfit(self, prices: List[int]) -> int:
        best, lowest = 0, 100000
        if len(prices) < 2:
            return best
        for price in prices:
            best = max(price - lowest, best)
            lowest = min(lowest, price)
        return best

122. 买卖股票的最佳时机 II

https://leetcode.cn/problems/best-time-to-buy-and-sell-stock-ii/description/?envType=study-plan-v2&envId=top-interview-150

这题有点所谓仿真思想了，也就是自己炒过股，知道低买高卖，然后自然想要去设置指针跟踪低点高点，不过高点当时就操作了，完全没必要跟踪，因此就一个指针 lowest 即可；此外还需要一个状态跟踪，就是当前是涨是跌的 is_up，剩下的就是标准的状态机思想。凡是涉及到拐点、升降变化的，大多要用状态机来捕捉拐点。

这道题一遍过，表扬自己注意了边界条件；但是也是因为题例给的好，如果没给的话也应该想到全升的情况。这道题和第一题的目标其实完全不同，它也是想要最大化利差，但不是一个全局 global 目标，而是可以多局部 local 求和的目标。如果还可以做空的话，是不是可以变成双向的遍历？

In [ ]:
class Solution:
    def maxProfit(self, prices: List[int]) -> int:
        n = len(prices)
        if n < 2:
            return 0
        total = 0
        is_up, lowest = False, 0
        for i in range(1, n):
            if prices[i] >= prices[i - 1]: # rising
                if not is_up:
                    is_up = True
                    lowest = i - 1
                if is_up and i == (n - 1):
                    total += prices[i] - prices[lowest] 
            else:
                if is_up:
                    is_up = False
                    total += prices[i - 1] - prices[lowest]      
        return total